This notebook implements a proof checker for the Fitch-style proof system for propositional logic defined in Section 2 of ["A fundamental non-classical logic"](https://arxiv.org/abs/2207.06993) by [Wesley H. Holliday](mailto:wesholliday@berkeley.edu).

# Well-formed formulas

Formulas are strings. We use single characters for propositional variables, ^ for conjunction, v for disjunction, ~ for negation, and parentheses for parentheses.

In [953]:
def wff(s):
    """
    Checks whether the string s is a well-formed formula and
    return the main connective (or 'prop' if s is a propositional variable)
    """

    if not type(s) is str:
        return False

    s = s.replace(" ", "")
        
    if len(s) == 1 and not s[0] in "()~^v":
        return True, "prop"

    if s.startswith("~"):
        sminus = s.removeprefix("~")
        if wff(sminus):
            return True, "~"

    if s.startswith("("):
            
        leftparen = 0
        rightparen = 0
        slength = len(s)

        for n in range(slength):

            if s[n] == "(":
                leftparen +=1

            if s[n] == ")":
                rightparen +=1

            if leftparen - rightparen == 1:
                
                if s[n] == "^" and wff(s[1:n]) and wff(s[n+1:-1]):
                    return True, "^"

                if s[n] == "v" and wff(s[1:n]) and wff(s[n+1:-1]):
                    return True, "v"

    return False

In [954]:
wff("(p ^ (q ^ (r v s)))")

(True, '^')

In [955]:
wff("(pp ^ (q ^ (r v s)))")

False

In [956]:
wff("~~(p v ~(q ^ r))")

(True, '~')

In [957]:
wff("~(~(p v ~(q ^ r))")

False

# Fitch-style proofs

A Fitch-style proof is a list of formulas and possibly other proofs, defined inductively in Section 2 of the paper.

In [958]:
def nospaces(p):
    newp = []
    for x in p:
        if type(x) is str:
            x = x.replace(" ","")
        if type(x) is list:
            x = nospaces(x)
        newp.append(x)
    return newp

In [959]:
nospaces(["((p ^ q) v (r ^ s))", ["(p ^ q)", "p", "(p v r)"], ["(r ^ s)", "r", "(p v r)"], "(p v r)"])

['((p^q)v(r^s))', ['(p^q)', 'p', '(pvr)'], ['(r^s)', 'r', '(pvr)'], '(pvr)']

To check whether p is a proof of "fundamental logic", run the following proof function on p.

For orthologic, set RAA = True.

For intuitionistic logic, set Reit = True.

For classical logic, set RAA = True and Reit = True.

For the "minimal" version of fundamental logic (cf. Johansson's minimal logic) in Theorem 4.23.4 of the paper, set Explosion = False.

In [960]:
def proof(p, RAA = None, Reit = None, reiterables = None, Explosion = None):
    """
    Checks whether the list p is a proof according to Section 2 of the paper;
    set RAA/Reit to True to allow the use of Reductio ad Absurdum/Reiteration;
    set Explosion to False for the "minimal" version of the fundamental logic.
    """
    p = nospaces(p)

    if RAA == None:
        RAA = False 

    if Reit == None:
        Reit = False 

    if reiterables == None:
        reiterables = []

    if Explosion == None:
        Explosion = True

    _proof(p, RAA, Reit, reiterables, Explosion)
        
def _proof(p, RAA, Reit, reiterables, Explosion):
    
    if not type(p) is list:
        print("A proof must be a list.")
        return False

    for x in p:
        if type(x) is str and not wff(x):
            print(f"{x} is not a well-formed formula.")
            return False

    # We regard a sequence consisting of one formula as a proof whose conclusion is its assumption.
    if len(p) == 1 and type(p[0]) is str: 
        print(f"{p} is a proof with assumption = conclusion.")
        return True

    if len(p) > 1:
        
        last = p[-1] 
        p_before_last = p[:-1] 
        
        if not _proof(p_before_last, RAA, Reit, reiterables, Explosion):
            if Reit == False:
                print(f"{p} is not a proof.")
            else:
                print(f"{p} is not a proof given reiterables = {reiterables}.")
            return False

        newreiterables = reiterables + [s for s in p_before_last if type(s) is str]
        
        if type(last) is list and _proof(last, RAA, Reit, newreiterables, Explosion):
            if Reit == False:
                print(f"{p} is a proof ending with a subproof.")
            else:
                print(f"{p} is a proof ending with a subproof given reiterables = {reiterables}.")
            return True

        if type(last) is str:
            if Reit == True and last in reiterables:
                print(f"{p} is a proof ending with Reiteration given reiterables = {reiterables}.")
                return True

            # We now search for a premise x from which last could be inferred by a rule
            for x in p_before_last:
                if type(x) is str:

                    if x.startswith("(" + last + "^"):
                        if Reit == False:
                            print(f"{p} is a proof ending with ^ Elim (left).")
                        else:
                            print(f"{p} is a proof ending with ^ Elim (left) given reiterables = {reiterables}.")
                        return True

                    if x.endswith("^" + last + ")"):
                        if Reit == False:
                            print(f"{p} is a proof ending with ^ Elim (right).")
                        else:
                            print(f"{p} is a proof ending with ^ Elim (right) given reiterables = {reiterables}.")
                        return True

                    if last.startswith("(" + x + "v"):
                        if Reit == False:
                            print(f"{p} is a proof ending with v Intro (right).")
                        else:
                            print(f"{p} is a proof ending with v Intro (right) given reiterables = {reiterables}.")
                        return True
        
                    if last.endswith("v" + x + ")"):
                        if Reit == False:
                            print(f"{p} is a proof ending with v Intro (left).")
                        else:
                            print(f"{p} is a proof ending with v Intro (left) given reiterables = {reiterables}.")
                        return True

                    # ^ Intro and ~ Elim have two premises, so we search for a second premise y
                    for y in p_before_last:
                        if type(y) is str and not x == y:

                            if last == "(" + x + "^" + y + ")":
                                if Reit == False:
                                    print(f"{p} is a proof ending with ^ Intro.")
                                else:
                                    print(f"{p} is a proof ending with ^ Intro given reiterables = {reiterables}")
                                return True

                            if Explosion == True and x == "~" + y:
                                if Reit == False:
                                    print(f"{p} is a proof ending with ~ Elim.") 
                                if Reit == True:
                                    print(f"{p} is a proof ending with ~ Elim given reiterables = {reiterables}.")
                                return True

            # The only remaining possibility is that the penultimate item in p is a subproof, 
            # in which case last could be obtained by ~ Intro (or RAA if permitted) or v Elim.
            penultimate = p_before_last[-1] 

            if type(penultimate) is list:
                
                p_before_penultimate = p_before_last[:-1]
                subproof_assumption = penultimate[0]
                subproof_conclusion = penultimate[-1]

                if type(subproof_assumption) is str and type(subproof_conclusion) is str:
                    if subproof_assumption == last.removeprefix("~"):
                        for x in p_before_penultimate:
                            if type(x) is str:
                                if subproof_conclusion == "~" + x or x == "~" + subproof_conclusion:
                                    if Reit == False:
                                        print(f"{p} is a proof ending with ~ Intro.")
                                    else:
                                        print(f"{p} is a proof ending with ~ Intro given reiterables = {reiterables}.")
                                    return True

                    if RAA == True and last == subproof_assumption.removeprefix("~"):
                        for x in p_before_penultimate:
                            if type(x) is str:
                                if subproof_conclusion == "~" + x or x == "~" + subproof_conclusion:
                                    if Reit == False:
                                        print(f"{p} is a proof ending with RAA.")
                                    else:
                                        print(f"{p} is a proof ending with RAA given reiterables = {reiterables}.")
                                    return True

                    # If the antepenultimate item in p is a subproof, then last could be obtained by v Elim
                    antepenultimate = p_before_penultimate[-1]

                    if type(antepenultimate) is list: 

                        p_before_antepenultimate = p_before_penultimate[:-1]
                        prior_subproof_assumption = antepenultimate[0]
                        prior_subproof_conclusion = antepenultimate[-1]

                        if type(prior_subproof_assumption) is str and type(prior_subproof_conclusion) is str:
                            for x in p_before_antepenultimate:
                                if type(x) is str:
                                    if x == "(" + prior_subproof_assumption + "v" + subproof_assumption + ")":
                                        if last == prior_subproof_conclusion and last == subproof_conclusion:
                                            
                                            if Reit == False:
                                                print(f"{p} is a proof ending with v Elim.")
                                            else:
                                                print(f"{p} is a proof ending with v Elim given reiterables = {reiterables}.")

                                            return True
    if Reit == False:
        print(f"{p} is not a proof.")
    else:
        print(f"{p} is not a proof given reiterables {reiterables}.")
    
    return False

# Examples

In [961]:
proof(['(p ^ q)','p','q','(q ^ p)'])

['(p^q)'] is a proof with assumption = conclusion.
['(p^q)', 'p'] is a proof ending with ^ Elim (left).
['(p^q)', 'p', 'q'] is a proof ending with ^ Elim (right).
['(p^q)', 'p', 'q', '(q^p)'] is a proof ending with ^ Intro.


In [962]:
proof(['(p ^ q)','p','q','(q ^ p))'])

(q^p)) is not a well-formed formula.


In [963]:
proof(["((p ^ q) v (r ^ s))", ["(p ^ q)", "p", "(p v r)"], ["(r ^ s)", "r", "(p v r)"], "(p v r)"])

['((p^q)v(r^s))'] is a proof with assumption = conclusion.
['(p^q)'] is a proof with assumption = conclusion.
['(p^q)', 'p'] is a proof ending with ^ Elim (left).
['(p^q)', 'p', '(pvr)'] is a proof ending with v Intro (right).
['((p^q)v(r^s))', ['(p^q)', 'p', '(pvr)']] is a proof ending with a subproof.
['(r^s)'] is a proof with assumption = conclusion.
['(r^s)', 'r'] is a proof ending with ^ Elim (left).
['(r^s)', 'r', '(pvr)'] is a proof ending with v Intro (left).
['((p^q)v(r^s))', ['(p^q)', 'p', '(pvr)'], ['(r^s)', 'r', '(pvr)']] is a proof ending with a subproof.
['((p^q)v(r^s))', ['(p^q)', 'p', '(pvr)'], ['(r^s)', 'r', '(pvr)'], '(pvr)'] is a proof ending with v Elim.


In [964]:
# A proof of distributivity in intuitionistic logic using reiteration
proof(["(p ^ (q v r))", "p", "(q v r)", ["q", "p", "(p ^ q)", "((p ^ q) v (p ^ r))"], ["r", "p", "(p ^ r)", "((p ^ q) v (p ^ r))"], "((p ^ q) v (p ^ r))"], Reit = True)

['(p^(qvr))'] is a proof with assumption = conclusion.
['(p^(qvr))', 'p'] is a proof ending with ^ Elim (left) given reiterables = [].
['(p^(qvr))', 'p', '(qvr)'] is a proof ending with ^ Elim (right) given reiterables = [].
['q'] is a proof with assumption = conclusion.
['q', 'p'] is a proof ending with Reiteration given reiterables = ['(p^(qvr))', 'p', '(qvr)'].
['q', 'p', '(p^q)'] is a proof ending with ^ Intro given reiterables = ['(p^(qvr))', 'p', '(qvr)']
['q', 'p', '(p^q)', '((p^q)v(p^r))'] is a proof ending with v Intro (right) given reiterables = ['(p^(qvr))', 'p', '(qvr)'].
['(p^(qvr))', 'p', '(qvr)', ['q', 'p', '(p^q)', '((p^q)v(p^r))']] is a proof ending with a subproof given reiterables = [].
['r'] is a proof with assumption = conclusion.
['r', 'p'] is a proof ending with Reiteration given reiterables = ['(p^(qvr))', 'p', '(qvr)'].
['r', 'p', '(p^r)'] is a proof ending with ^ Intro given reiterables = ['(p^(qvr))', 'p', '(qvr)']
['r', 'p', '(p^r)', '((p^q)v(p^r))'] is a pr

In [965]:
# The intuitionistic proof above is not a proof in the fundamental logic
proof(["(p ^ (q v r))", "p", "(q v r)", ["q", "p", "(p ^ q)", "((p ^ q) v (p ^ r))"], ["r", "p", "(p ^ r)", "((p ^ q) v (p ^ r))"], "((p ^ q) v (p ^ r))"])

['(p^(qvr))'] is a proof with assumption = conclusion.
['(p^(qvr))', 'p'] is a proof ending with ^ Elim (left).
['(p^(qvr))', 'p', '(qvr)'] is a proof ending with ^ Elim (right).
['q'] is a proof with assumption = conclusion.
['q', 'p'] is not a proof.
['q', 'p', '(p^q)'] is not a proof.
['q', 'p', '(p^q)', '((p^q)v(p^r))'] is not a proof.
['(p^(qvr))', 'p', '(qvr)', ['q', 'p', '(p^q)', '((p^q)v(p^r))']] is not a proof.
['(p^(qvr))', 'p', '(qvr)', ['q', 'p', '(p^q)', '((p^q)v(p^r))'], ['r', 'p', '(p^r)', '((p^q)v(p^r))']] is not a proof.
['(p^(qvr))', 'p', '(qvr)', ['q', 'p', '(p^q)', '((p^q)v(p^r))'], ['r', 'p', '(p^r)', '((p^q)v(p^r))'], '((p^q)v(p^r))'] is not a proof.


In [966]:
# An incorrect attempt to reiterate from one subproof to another
proof(["(p v q)", ["p"], ["q","p"], "p"], Reit = True)

['(pvq)'] is a proof with assumption = conclusion.
['p'] is a proof with assumption = conclusion.
['(pvq)', ['p']] is a proof ending with a subproof given reiterables = [].
['q'] is a proof with assumption = conclusion.
['q', 'p'] is not a proof given reiterables ['(pvq)'].
['(pvq)', ['p'], ['q', 'p']] is not a proof given reiterables [].
['(pvq)', ['p'], ['q', 'p'], 'p'] is not a proof given reiterables = [].


In [967]:
# ~ is antitone
proof(["~p", ["(p ^ q)", "p"], "~(p ^ q)"])

['~p'] is a proof with assumption = conclusion.
['(p^q)'] is a proof with assumption = conclusion.
['(p^q)', 'p'] is a proof ending with ^ Elim (left).
['~p', ['(p^q)', 'p']] is a proof ending with a subproof.
['~p', ['(p^q)', 'p'], '~(p^q)'] is a proof ending with ~ Intro.


In [968]:
# ~ is a semicomplementation
proof(["(p ^ ~p)", "p", "~p", "q"])

['(p^~p)'] is a proof with assumption = conclusion.
['(p^~p)', 'p'] is a proof ending with ^ Elim (left).
['(p^~p)', 'p', '~p'] is a proof ending with ^ Elim (right).
['(p^~p)', 'p', '~p', 'q'] is a proof ending with ~ Elim.


In [969]:
proof(["(p ^ ~p)", "p", "~p", "q"], Explosion = False)

['(p^~p)'] is a proof with assumption = conclusion.
['(p^~p)', 'p'] is a proof ending with ^ Elim (left).
['(p^~p)', 'p', '~p'] is a proof ending with ^ Elim (right).
['(p^~p)', 'p', '~p', 'q'] is not a proof.


In [970]:
# Double negation introduction
proof(["p", ["~p"], "~~p"])

['p'] is a proof with assumption = conclusion.
['~p'] is a proof with assumption = conclusion.
['p', ['~p']] is a proof ending with a subproof.
['p', ['~p'], '~~p'] is a proof ending with ~ Intro.


In [971]:
# Double negation elimination using Reductio
proof(["~~p", ["~p"], "p"], RAA = True)

['~~p'] is a proof with assumption = conclusion.
['~p'] is a proof with assumption = conclusion.
['~~p', ['~p']] is a proof ending with a subproof.
['~~p', ['~p'], 'p'] is a proof ending with RAA.


In [972]:
proof(["~~p", ["~p"], "p"])

['~~p'] is a proof with assumption = conclusion.
['~p'] is a proof with assumption = conclusion.
['~~p', ['~p']] is a proof ending with a subproof.
['~~p', ['~p'], 'p'] is not a proof.
